In [1]:
from qdrant_client.http import models
from internal_assistant_core import settings, qdrant_client, vectorstoreQ

🔗 Connecting to Qdrant at https://qdrant-con.whitecliff-cbbdbf53.southeastasia.azurecontainerapps.io/
Health check: 200 healthz check passed
✅ Connected to Qdrant. Collections: 0
⚠️ Warning: Failed to initialize VectorStore: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `internal-docs-index` doesn\'t exist!"},"time":7.04e-6}'
[MEMORY INIT] Connecting to Redis: documentWorkflow.redis.cache.windows.net:6380
✅ Redis connection successful
[MEMORY INIT] Connecting to Cosmos DB: https://documentworkflow.documents.azure.com:443/
[MEMORY INIT] Creating/getting database: internal_assistant
[MEMORY INIT] Creating/getting container: conversation_history
✅ Cosmos DB connection successful
✅ Memory Manager initialized with module separation


In [5]:
from qdrant_client import QdrantClient
import urllib3

urllib3.disable_warnings()  # untuk menghindari SSL warning

print(f"🔗 Connecting to Qdrant at {settings.qdrant_url}")

try:
    qdrant_client = QdrantClient(
        url=settings.qdrant_url,
        api_key=settings.qdrant_api_key or None,
        timeout=60,
        prefer_grpc=False,   # gunakan REST API
        https=True,          # pastikan gunakan HTTPS
        verify=False,
        port=443              # ⚠️ abaikan SSL verify error
    )

    collections = qdrant_client.get_collections()
    print(f"✅ Connected to Qdrant. Collections: {len(collections.collections)}")

except Exception as e:
    print(f"⚠️ Warning: Failed to initialize VectorStore: {e}")


🔗 Connecting to Qdrant at https://qdrant-con.whitecliff-cbbdbf53.southeastasia.azurecontainerapps.io/
✅ Connected to Qdrant. Collections: 0


In [6]:
from qdrant_client.http import models

# Check if client exists
if qdrant_client is None:
    print("❌ Qdrant client not initialized. Check connection first.")
else:
    collection_name = settings.qdrant_collection

    # Hapus collection lama kalau ada, lalu buat baru
    if qdrant_client.collection_exists(collection_name):
        qdrant_client.delete_collection(collection_name)

    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=3072,
            distance=models.Distance.COSINE
        ),
        hnsw_config=models.HnswConfigDiff(
            m=16,
            ef_construct=400,
            full_scan_threshold=10000
        ),
        optimizers_config=models.OptimizersConfigDiff(
            default_segment_number=2
        )
    )

    print(f"Collection '{collection_name}' berhasil dibuat dengan konfigurasi mirip Azure AI Search 🚀")


Collection 'internal-docs-index' berhasil dibuat dengan konfigurasi mirip Azure AI Search 🚀


In [3]:
# # Nama collection (setara "internal-docs-index" di Azure)
collection_name = settings.qdrant_collection

# Hapus collection lama kalau ada, lalu buat baru
if qdrant_client.collection_exists(collection_name):
    qdrant_client.delete_collection(collection_name)

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=3072,                         # Dimension embedding (text-embedding-3-large)
        distance=models.Distance.COSINE    # Metric cosine similarity
    ),
    hnsw_config=models.HnswConfigDiff(
        m=16,
        ef_construct=400,
        full_scan_threshold=10000
    ),
    optimizers_config=models.OptimizersConfigDiff(
        default_segment_number=2
    )
)

print(f"Collection '{collection_name}' berhasil dibuat dengan konfigurasi mirip Azure AI Search 🚀")



AttributeError: 'NoneType' object has no attribute 'collection_exists'

In [ ]:
print(qdrant_client.get_collection("internal-docs-index"))
print(qdrant_client.count("internal-docs-index"))


In [2]:
from rag_modul import process_and_index_docs

# Reindex semua dokumen SOP
result = process_and_index_docs(prefix="sop/")

print(f"Indexed: {result['indexed']} documents")
print(f"Total chunks: {result['total_chunks']}")
print(f"Skipped: {result['skipped']}")
print(f"Errors: {len(result['errors'])}")

Starting to process documents with prefix: 'sop/'
Processing: sop/0. Employee Handbook - Local Policies Indonesia.pdf
✅ Document Intelligence extracted 39 pages
📊 Found 13 raw tables, checking for continuations...
  ✓ First row appears to be data
✓ Table 1 detected as continuation of table 0
  ✓ Column types match: 1.00
✓ Table 2 detected as continuation of table 0
  Merging 3 tables into one
✓ Merged 3 tables
  ✓ First row appears to be data
✓ Table 6 detected as continuation of table 5
  Merging 2 tables into one
✓ Merged 2 tables
  ✓ Column types match: 1.00
✓ Table 9 detected as continuation of table 8
  Merging 2 tables into one
✓ Merged 2 tables
📊 After merging: 9 tables
✓ Table 0: Extracted 60 rows, 2 columns
✓ Table 1: Extracted 26 rows, 2 columns
✓ Table 2: Extracted 3 rows, 7 columns
✓ Table 3: Extracted 7 rows, 2 columns
✓ Table 4: Extracted 8 rows, 2 columns
✓ Table 5: Extracted 9 rows, 3 columns
✓ Table 6: Extracted 4 rows, 3 columns
✓ Table 7: Extracted 18 rows, 3 columns

In [ ]:
print(client.get_collections())

In [ ]:
from internal_assistant_core import settings, vectorstoreQ, qdrant_client

# Debug settings
print(f"QDRANT_URL: {settings.qdrant_url}")
print(f"QDRANT_API_KEY: {settings.qdrant_api_key}")
print(f"QDRANT_COLLECTION: {settings.qdrant_collection}")

# Debug client
print(f"Qdrant Client: {qdrant_client}")
print(f"VectorstoreQ: {vectorstoreQ}")
print(f"VectorstoreQ type: {type(vectorstoreQ)}")

In [ ]:
# Test Document Extraction
from internal_assistant_core import blob_container
from rag_modul import _extract_text_with_docint
import json

def test_document_extraction(blob_name: str):
    """Test ekstraksi dokumen dan tampilkan hasilnya"""
    try:
        # Download dokumen
        blob_client = blob_container.get_blob_client(blob_name)
        content_bytes = blob_client.download_blob().readall()
        
        # Ekstraksi dengan Document Intelligence
        doc_data = _extract_text_with_docint(content_bytes)
        
        # Save hasil ke file JSON
        filename = f"ekstraksi_{blob_name.replace('/', '_').replace('.pdf', '')}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(doc_data, f, indent=2, ensure_ascii=False)
        
        # Print summary
        print(f"Dokumen: {blob_name}")
        print(f"Sections: {len(doc_data['sections'])}")
        print(f"Tables: {len(doc_data['raw_tables'])}")
        print(f"Saved to: {filename}")
        
        # Show first few sections
        for i, section in enumerate(doc_data['sections'][:3]):
            print(f"\nSection {i+1}: {section['header']}")
            print(f"  Type: {section['type']}")
            print(f"  Content parts: {len(section['content_parts'])}")
            print(f"  Total tokens: {section['total_tokens']}")
            
        return doc_data
        
    except Exception as e:
        print(f"Error: {e}")
        return None

def list_available_documents():
    """List semua dokumen yang ada di blob storage"""
    try:
        print("📂 Available documents:")
        blob_list = list(blob_container.list_blobs())
        
        if not blob_list:
            print("❌ No documents found in blob storage")
            return []
            
        for i, blob in enumerate(blob_list, 1):
            print(f"  {i}. {blob.name}")
            
        return [blob.name for blob in blob_list]
        
    except Exception as e:
        print(f"Error listing documents: {e}")
        return []

if __name__ == "__main__":
    # List available documents first
    docs = list_available_documents()
    
    if docs:
        # Test dengan dokumen pertama yang ada
        print(f"\n🧪 Testing extraction with: {docs[0]}")
        test_document_extraction(docs[0])
    else:
        print("No documents to test with")

In [4]:
import requests

resp = requests.get(settings.qdrant_url )
print(resp.status_code, resp.text)


200 {"title":"qdrant - vector search engine","version":"1.15.4","commit":"20db14f87c861f3958ad50382cf0b69396e40c10"}


In [ ]:
# # Nama collection (setara "internal-docs-index" di Azure)
collection_name = settings.qdrant_collection

# Hapus collection lama kalau ada, lalu buat baru
if qdrant_client.collection_exists(collection_name):
    qdrant_client.delete_collection(collection_name)

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=3072,                         # Dimension embedding (text-embedding-3-large)
        distance=models.Distance.COSINE    # Metric cosine similarity
    ),
    hnsw_config=models.HnswConfigDiff(
        m=16,
        ef_construct=400,
        full_scan_threshold=10000
    ),
    optimizers_config=models.OptimizersConfigDiff(
        default_segment_number=2
    )
)

print(f"Collection '{collection_name}' berhasil dibuat dengan konfigurasi mirip Azure AI Search 🚀")

